# Section 1: Data Loading

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('binary_class').getOrCreate()


Imported the sparksession from psyspark.

In [0]:
from pyspark.ml.feature import VectorAssembler

Imported vector assembler package from psyspark to work with vectors.

In [0]:
from pyspark.ml.classification import LogisticRegression

Imported LogisticRegression package from psyspark to run regressions.

In [0]:
df = spark.read.csv('/FileStore/tables/Default2.csv', inferSchema=True, header=True)

Read the csv file and created a data frame with Default2 csv and infered header/schema.

In [0]:
print("Rows = {}, Columns = {}".format(df.count(), len(df.columns)))

Rows = 10000, Columns = 5


Used print to display the number of rows and columns.

In [0]:
df.show(15, truncate = False)

+--------+-------+-------+-----------+-----------+
|RecordID|default|student|balance    |income     |
+--------+-------+-------+-----------+-----------+
|1137    |Yes    |No     |2499.02    |51504.29396|
|2       |No     |Yes    |817.1804066|12106.1347 |
|6076    |Yes    |PT     |2413.32    |38540.57271|
|9874    |No     |No     |2391.01    |50302.90956|
|5       |No     |PT     |785.6558829|38463.49588|
|6       |No     |Yes    |919.5885305|7491.558572|
|6335    |Yes    |PT     |2343.8     |51095.29393|
|8       |No     |Yes    |808.6675043|17600.45134|
|9894    |Yes    |PT     |2288.41    |52043.56905|
|8265    |Yes    |PT     |2236.76    |37113.88307|
|11      |No     |Yes    |0.0        |21871.07309|
|12      |No     |Yes    |1220.583753|13268.56222|
|3190    |Yes    |PT     |2228.47    |27438.34899|
|1360    |Yes    |PT     |2220.97    |40725.09621|
|9979    |Yes    |PT     |2202.46    |47287.25711|
+--------+-------+-------+-----------+-----------+
only showing top 15 rows



Used df.show(15) to get the top 15 rows.

# Section 2: Data Cleansing

In [0]:
df_clean = df.dropna()

Created an object called dfna that has the df without the empty values.

In [0]:
df_clean.show(15, truncate = False)

+--------+-------+-------+-----------+-----------+
|RecordID|default|student|balance    |income     |
+--------+-------+-------+-----------+-----------+
|1137    |Yes    |No     |2499.02    |51504.29396|
|2       |No     |Yes    |817.1804066|12106.1347 |
|6076    |Yes    |PT     |2413.32    |38540.57271|
|9874    |No     |No     |2391.01    |50302.90956|
|5       |No     |PT     |785.6558829|38463.49588|
|6       |No     |Yes    |919.5885305|7491.558572|
|6335    |Yes    |PT     |2343.8     |51095.29393|
|8       |No     |Yes    |808.6675043|17600.45134|
|9894    |Yes    |PT     |2288.41    |52043.56905|
|8265    |Yes    |PT     |2236.76    |37113.88307|
|11      |No     |Yes    |0.0        |21871.07309|
|12      |No     |Yes    |1220.583753|13268.56222|
|3190    |Yes    |PT     |2228.47    |27438.34899|
|1360    |Yes    |PT     |2220.97    |40725.09621|
|9979    |Yes    |PT     |2202.46    |47287.25711|
+--------+-------+-------+-----------+-----------+
only showing top 15 rows



Showing the top 15 of the cleaned data frame.

# Section 3: Simple Descriptive Statistics

In [0]:
df_avg = df_clean.groupBy(['default', 'student']).mean('balance','income').orderBy('student').show(truncate = False)

+-------+-------+------------------+------------------+
|default|student|avg(balance)      |avg(income)       |
+-------+-------+------------------+------------------+
|No     |No     |726.8519629742985 |39994.22243017542 |
|Yes    |No     |1524.368677685951 |40577.940826363636|
|Yes    |PT     |1897.739284682353 |40692.112921529406|
|No     |PT     |1132.0960709586857|40018.28049266447 |
|Yes    |Yes    |1860.3790920472436|18243.508299598423|
|No     |Yes    |948.3809424503651 |17935.953138127275|
+-------+-------+------------------+------------------+



Created an object named df_avg that used the data from the clean df. Grouped it by default and student. Then, got the mean of the balance and income along with ordering it by the students to make it easier to compare the income/balance to defaults. Ended it with show to make the df_avg visible

# Do the people who defaulted have a higher or lower average income compared to those who did not default?
The people who default on average have a higher income. It appears that those who default make at least $200 more.

In [0]:
corr = df_clean.corr('balance', 'income')
print('Correlation between balance and income: ', corr)

Correlation between balance and income:  -0.1521139758423617


Created an object called corr that uses the clean dataframe. The corr function gets the correlation between blance and income. The I used a print with a string and the corr object to display the result.

# Section 4: Data recording

In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

Imported from pyspark the onehot coder, string index, and vectorassembler to dummy code, use vectors, and create numbers for catergoical columns.

In [0]:
student_indexer = StringIndexer(inputCol = "student", outputCol="student_index").fit(df_clean)
df_clean1 = student_indexer.transform(df_clean)
df_clean1.show(truncate = False)

+--------+-------+-------+-----------+-----------+-------------+
|RecordID|default|student|balance    |income     |student_index|
+--------+-------+-------+-----------+-----------+-------------+
|1137    |Yes    |No     |2499.02    |51504.29396|0.0          |
|2       |No     |Yes    |817.1804066|12106.1347 |1.0          |
|6076    |Yes    |PT     |2413.32    |38540.57271|2.0          |
|9874    |No     |No     |2391.01    |50302.90956|0.0          |
|5       |No     |PT     |785.6558829|38463.49588|2.0          |
|6       |No     |Yes    |919.5885305|7491.558572|1.0          |
|6335    |Yes    |PT     |2343.8     |51095.29393|2.0          |
|8       |No     |Yes    |808.6675043|17600.45134|1.0          |
|9894    |Yes    |PT     |2288.41    |52043.56905|2.0          |
|8265    |Yes    |PT     |2236.76    |37113.88307|2.0          |
|11      |No     |Yes    |0.0        |21871.07309|1.0          |
|12      |No     |Yes    |1220.583753|13268.56222|1.0          |
|3190    |Yes    |PT     

Student is a string. In order to get it as one hot encoder we first have to convert it to a numeric format using string_indexer with student as the column and student_index as the new column.

In [0]:
student_encoder = OneHotEncoder(inputCol="student_index", outputCol="student_vec").fit(df_clean1)
df_clean2 = student_encoder.transform(df_clean1)
df_clean2.show(truncate = False)

+--------+-------+-------+-----------+-----------+-------------+-------------+
|RecordID|default|student|balance    |income     |student_index|student_vec  |
+--------+-------+-------+-----------+-----------+-------------+-------------+
|1137    |Yes    |No     |2499.02    |51504.29396|0.0          |(2,[0],[1.0])|
|2       |No     |Yes    |817.1804066|12106.1347 |1.0          |(2,[1],[1.0])|
|6076    |Yes    |PT     |2413.32    |38540.57271|2.0          |(2,[],[])    |
|9874    |No     |No     |2391.01    |50302.90956|0.0          |(2,[0],[1.0])|
|5       |No     |PT     |785.6558829|38463.49588|2.0          |(2,[],[])    |
|6       |No     |Yes    |919.5885305|7491.558572|1.0          |(2,[1],[1.0])|
|6335    |Yes    |PT     |2343.8     |51095.29393|2.0          |(2,[],[])    |
|8       |No     |Yes    |808.6675043|17600.45134|1.0          |(2,[1],[1.0])|
|9894    |Yes    |PT     |2288.41    |52043.56905|2.0          |(2,[],[])    |
|8265    |Yes    |PT     |2236.76    |37113.88307|2.

After student was turned into a numerical column as stundent_index, we now can use the onehote code to dummy code and get the vector response.

In [0]:
default_indexer = StringIndexer(inputCol = "default", outputCol="default_index").fit(df_clean2)
df_clean3 = default_indexer.transform(df_clean2)
df_clean3.show(truncate = False)

+--------+-------+-------+-----------+-----------+-------------+-------------+-------------+
|RecordID|default|student|balance    |income     |student_index|student_vec  |default_index|
+--------+-------+-------+-----------+-----------+-------------+-------------+-------------+
|1137    |Yes    |No     |2499.02    |51504.29396|0.0          |(2,[0],[1.0])|1.0          |
|2       |No     |Yes    |817.1804066|12106.1347 |1.0          |(2,[1],[1.0])|0.0          |
|6076    |Yes    |PT     |2413.32    |38540.57271|2.0          |(2,[],[])    |1.0          |
|9874    |No     |No     |2391.01    |50302.90956|0.0          |(2,[0],[1.0])|0.0          |
|5       |No     |PT     |785.6558829|38463.49588|2.0          |(2,[],[])    |0.0          |
|6       |No     |Yes    |919.5885305|7491.558572|1.0          |(2,[1],[1.0])|0.0          |
|6335    |Yes    |PT     |2343.8     |51095.29393|2.0          |(2,[],[])    |1.0          |
|8       |No     |Yes    |808.6675043|17600.45134|1.0          |(2,[1]

Used string indexer to get the default column as a numerical value.

# Section 5: Modeling

In [0]:
df_assembler = VectorAssembler(inputCols=['balance', 'income', 'student_index', 'student_vec'], outputCol="features")

Created a vector with the columns balance, income, student index, and student vec. The output is the features column and the object is called df assembler.

In [0]:
df_vec = df_assembler.transform(df_clean3)

Used the vector to created a new data frame called df_vec that has transformed the old data frame to add in the features column.

In [0]:
model_df=df_vec.select(['features','default_index'])

Created an object that uses the feautures column and default index.

In [0]:
training_df,test_df=model_df.randomSplit([0.70,0.30])

Split the data 70/30 and named the 70 as training_df and the test_df as 30.

In [0]:
log_reg=LogisticRegression(featuresCol ='features', labelCol = 'default_index').fit(training_df)

Ran the logist regression and put it in the object log_reg. The default_index was the label and fit it with the training data to run the regression.

In [0]:
training_df.columns

Out[135]: ['features', 'default_index']

Selected the columns to check that there were two columns in the training data frame.

In [0]:
print("Intercept: {}".format(log_reg.intercept))

Intercept: -10.687085038747885


Using the regression that was just ran, pulled the intercept from it.

In [0]:
print("Variables: {}".format(training_df.schema["features"].metadata))

Variables: {'ml_attr': {'attrs': {'numeric': [{'idx': 0, 'name': 'balance'}, {'idx': 1, 'name': 'income'}], 'nominal': [{'vals': ['No', 'Yes', 'PT'], 'idx': 2, 'name': 'student_index'}], 'binary': [{'idx': 3, 'name': 'student_vec_No'}, {'idx': 4, 'name': 'student_vec_Yes'}]}, 'num_attrs': 5}}


Printed the variables and their schema to know which order the coefficients will go.

In [0]:
print("Coefficients ({}): {}".format(len(log_reg.coefficients), log_reg.coefficients))

Coefficients (5): [0.005474048954667918,4.396622154248527e-06,0.06697436391018696,0.13942636245475643,-0.3968096018585849]


Printed the coefficients and the number of them.

The logistic regression equation is: 
$$p(default = 1) = \frac{e^{-10.6871 + 0.0055\*balance + 4.3966e^{-6}\*income+0.0670\*student\\_index\\ \= Yes+ 0.1394\*student\\_index\\ \= PT- 0.3968\*student\\_index\\ \= No}}{1+e^{-10.6871 + 0.0055\*balance + 4.3966e^{-6}\*income+0.0670\*student\\_index\\ \= Yes+ 0.1394\*student\\_index\\ \= PT- 0.3968\*student\\_index\\ \= No}}$$

Created the logistic regression equation is LaTex form from the regression that I ran with the coeffcients and intercept.

In [0]:
lr_summary=log_reg.summary
lr_summary.accuracy

Out[143]: 0.9733824154042192

Shows the accuracy of the regression in the training_df.

In [0]:
lr_summary.areaUnderROC

Out[144]: 0.9447427990025711

Shows the ROC from the training_df regression.

In [0]:
print(lr_summary.precisionByLabel)

[0.976867816091954, 0.7378640776699029]


Shows the precision of the training_df regression.

In [0]:
print(lr_summary.recallByLabel)

[0.9960445355991796, 0.3206751054852321]


Shows the recall of the training_df regression.

In [0]:
predictions = log_reg.transform(test_df)

Using the regression, we now use it after creating it from the training data to verify the accuracy with the untouched test_df.

In [0]:
predictions.show(10)

+--------------------+-------------+--------------------+--------------------+----------+
|            features|default_index|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|(5,[1,2],[24565.5...|          0.0|[10.4451307743793...|[0.99997091127347...|       0.0|
|(5,[1,2],[36302.5...|          0.0|[10.3935278101329...|[0.99996937085158...|       0.0|
|(5,[1,2],[47895.8...|          0.0|[10.3425565539206...|[0.99996776922399...|       0.0|
|(5,[1,3],[16601.6...|          0.0|[10.4746675588243...|[0.99997175787220...|       0.0|
|(5,[1,3],[18063.8...|          0.0|[10.4682385845376...|[0.99997157572456...|       0.0|
|(5,[1,3],[18593.9...|          0.0|[10.4659082588130...|[0.99997150941140...|       0.0|
|(5,[1,3],[22074.4...|          0.0|[10.4506054682918...|[0.99997107008562...|       0.0|
|(5,[1,3],[23562.2...|          0.0|[10.4440641873923...|[0.99997088023217...|       0.0|
|(5,[1,3],

Displays the top 10 predictions from above.

In [0]:
model_predictions = log_reg.transform(test_df)
model_predictions = log_reg.evaluate(test_df)

Transforms the test_df and then evaluates it withing the object model_predictions.

In [0]:
model_predictions.accuracy

Out[153]: 0.9733333333333334

The accuracy of the logisitic regression on the test dataframe.

In [0]:
model_predictions.areaUnderROC

Out[154]: 0.9612201013314491

The ROC of the logisitic regression on the test dataframe.

In [0]:
print(model_predictions.recallByLabel)

[0.9968186638388123, 0.28125]


The recall of the logisitic regression on the test dataframe.

In [0]:
print(model_predictions.precisionByLabel)

[0.9761163032191069, 0.75]


The precisions of the logisitic regression on the test dataframe.